### This time start without sampling and test with LSTM

Without sampling usually failed bz it is unbalanced class dataset and you need to oversampling and undersampling.

Decision Tree ( av-precison = 0.258728) without sampling  
DNN-LSTM ( av-precison = 0.25872) without sampling

# Test without oversmapling

In [ ]:
import os 
import numpy as np 
import pandas as pd

In [ ]:
def preprocess_the_whole_stage(folder_path):
    # create an empty list to store the DataFrames
    dfs = []

    # loop over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            # extract the ID from the filename (assuming the filename is in the format "ID.csv")
            file_id = os.path.splitext(filename)[0]

            # read the CSV file into a DataFrame and add the ID as a new column
            df = pd.read_csv(os.path.join(folder_path, filename))
            df.insert(0, 'ID', file_id)

            # append the DataFrame to the list
            dfs.append(df)

    # concatenate all DataFrames into a single DataFrame
    full_dataset = pd.concat(dfs)
    
    # print the resulting DataFrame
    print(full_dataset.head())
    
    # Add all zero class
    condition = (full_dataset.StartHesitation == 0) & (full_dataset.Turn == 0) & (full_dataset.Walking == 0)
    condition_2 = (full_dataset.StartHesitation == 1) | (full_dataset.Turn == 1) | (full_dataset.Walking == 1)
    
    full_dataset.loc[condition, 'All_zero'] = 1
    full_dataset.loc[condition_2, 'All_zero'] = 0
    print(full_dataset.head())
    
    print("Cleaning the Dataset")
    if 'Valid' in full_dataset.columns:
        remove_col = ['ID', 'Valid', 'Task']
        
    else:
        remove_col = ['ID']
    print(f"The remove columns : {remove_col}")
    clean_dataset = full_dataset.drop(full_dataset[remove_col],axis=1)
    print(clean_dataset.head())
    
    # search duplication
    print(f"Search for Duplication : {clean_dataset.duplicated().sum()}")
    clean_dataset.drop_duplicates(inplace=True)
    print(f"Search for Duplication : {clean_dataset.duplicated().sum()}")
    print(clean_dataset.head())
    
    print("Checking conditon")
    condition = (clean_dataset.StartHesitation == 0) & (clean_dataset.Turn == 0 ) & (clean_dataset.Walking == 0)
    total_zero = clean_dataset[condition].shape[0]
    print(f"Total number where three class are zero: {total_zero}")
    All_zero = clean_dataset[clean_dataset.All_zero == 1].shape[0]
    print(f"Total number of All_zero class: {All_zero}")
    print(f"Is all zero and Total number zero are equal :{All_zero == total_zero}")
    a = clean_dataset[clean_dataset.StartHesitation == 1].shape[0]
    print(f"The number of Class Start Hesitation :{a}")
    b = clean_dataset[clean_dataset.Walking == 1].shape[0]
    print(f"The number of Class Walking :{b}")
    c = clean_dataset[clean_dataset.Turn == 1].shape[0]
    print(f"The number of Class Turn : {c}")
    print(f"Is the toatl number of sample equal to all Four class combine :"
         f"{clean_dataset.shape[0] == a + b + c + All_zero}")
    
    return clean_dataset


In [ ]:
def oversampling_and_split(clean_dataset):
    feature_col = ['Time', 'AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']
    
    feature_dataset = clean_dataset[feature_col]
    label_dataset = clean_dataset[label_col]
    print(f"The Feature :{feature_dataset.shape}, \n"
          f"The label {label_dataset.shape}")
    
    
    def check_all_four_class_condition(df):
        print(f"Check all four check condiiton in {df}")
        a = df[df.StartHesitation == 1].shape[0]
        b = df[df.Turn == 1].shape[0]
        c = df[df.Walking ==1].shape[0]
        d = df[df.All_zero == 1].shape[0]
        print(f"Number of Start Hesitation : {a}, \n"
              f"Number of Turn : {b}, \n"  
              f"Number of Walking : {c}, \n"
              f"Number of All_zero : {d}")
        print("Is Number of All four class is equal to total sampling :",
             df.shape[0] == a + b + c + d)

    
    oversampling_dataset = pd.concat([feature_dataset,label_dataset], 
                                     ignore_index= False, sort=False, axis=1)
    
    print(f"The shape of oversampling dataset is : {oversampling_dataset.shape[0]}")
    print(f"The number of duplication in dataset : {oversampling_dataset.duplicated().sum()}")
    # Drop duplication
    oversampling_dataset.drop_duplicates(inplace=True)
    print(f"The shape of oversampling after remove duplication :{oversampling_dataset.shape}")
    
    # 60% Train Data, 20% Validation Data, 20% Test Data
    # 80% Set Data(60% rain Data, 20% Validation Data) , 20% Test Data
    
    from sklearn.model_selection import train_test_split
    import random 
    random_seed = 54

    set_data, test_data = train_test_split(oversampling_dataset, test_size=0.2, random_state=True)
    print(f"The set data shape : {set_data.shape}\n"
          f"The test data shape : {test_data.shape}\n"
          f"Is the dataset still in range : "
          f"{oversampling_dataset.shape[0] == set_data.shape[0] + test_data.shape[0]}")
    
    print(f"Again Search for duplicaiton : \n "
          f"Set Data :{set_data.duplicated().sum()} \n"
          f"Test Data :{test_data.duplicated().sum()}")
    
    check_all_four_class_condition(set_data)
    check_all_four_class_condition(test_data)
    print("All task are finish")
    
    return set_data, test_data
    

In [ ]:
def preprocessing_the_dataset(df):
    def check_skewness(df):
    # this can check relation between each column
        skew_limit=0.75
        skew_value=df[df.columns].skew()
        #print(skew_value)
        skew_col=skew_value[abs(skew_value)>skew_limit]
        cols=skew_col.index
        return cols

    import random 
    random_seed = 54
    
    feature_col = ['Time','AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']

    # make feature and label
    feature_dataset = df[feature_col]
    label_dataset = df[label_col]
    
    # check skewness and powertransform
    skew_columns = check_skewness(feature_dataset)
    print(skew_columns)
    
    print("Power Transform start")
    from sklearn.preprocessing import PowerTransformer
    pt=PowerTransformer(standardize=False)  
    feature_dataset[skew_columns] = pt.fit_transform(feature_dataset[skew_columns])
    
    print("Standardization start")
    # Change features data to 0 and 1
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    feature_dataset=sc.fit_transform(feature_dataset)
    
    print("Train test split begin")
    from sklearn.model_selection import train_test_split
    train_feature, valid_feature, train_label, valid_label = train_test_split(feature_dataset, label_dataset, test_size=0.2, random_state=True)
    
    train_feature = np.array(train_feature) 
    valid_feature = np.array(valid_feature)
    train_label  = np.array(train_label)
    valid_label = np.array(valid_label)
    print("All task are finish")
    
    return train_feature, valid_feature, train_label, valid_label
    

In [ ]:
# specify the folder path
defog_path = "Data/train/defog"
clean_defog_dataset = preprocess_the_whole_stage(defog_path)
tdcsfog_path = "Data/train/tdcsfog"
clean_tdcsfog_dataset= preprocess_the_whole_stage(tdcsfog_path)


In [ ]:
clean_defog_dataset.head()

In [ ]:
clean_tdcsfog_dataset.head()

In [ ]:
clean_dataset = pd.concat([clean_tdcsfog_dataset, clean_defog_dataset ], 
                           ignore_index= True, sort=False, axis=0)

In [ ]:
set_data,test_data = oversampling_and_split(clean_dataset)

In [ ]:
train_feature, valid_feature, train_label, valid_label = preprocessing_the_dataset(set_data)
print(f"{train_feature.shape} , {train_label.shape} , {valid_feature.shape} , {valid_label.shape}, {test_data.shape}")

In [ ]:
def preprocessing_test_dataset(df):

    import random 
    random_seed = 54
    
    feature_col = ['Time','AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking']

    # make feature and label
    feature_dataset = df[feature_col]
    label_dataset = df[label_col]
    
    def check_skewness(df):
    # this can check relation between each column
        skew_limit=0.75
        skew_value=df[df.columns].skew()
        #print(skew_value)
        skew_col=skew_value[abs(skew_value)>skew_limit]
        cols=skew_col.index
        return cols
    
    # check skewness and powertransform
    skew_columns = check_skewness(feature_dataset)
    print(skew_columns)
    
    print("Power Transform start")
    from sklearn.preprocessing import PowerTransformer
    pt=PowerTransformer(standardize=False)  
    feature_dataset[skew_columns] = pt.fit_transform(feature_dataset[skew_columns])
    
    print("Standardization start")
    # Change features data to 0 and 1
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    feature_dataset=sc.fit_transform(feature_dataset)

    

    feature_dataset = np.array(feature_dataset) 
    label_dataset  = np.array(label_dataset)
    print("All task are finish")
    
    return feature_dataset, label_dataset
    

In [ ]:
test_feature, test_label = preprocessing_test_dataset(test_data)

In [ ]:
load_data_path = 'Time_no_sampling'

In [ ]:
np.save(f'Data/{load_data_path}/train_feature.npy',train_feature) 
np.save(f'Data/{load_data_path}/valid_feature.npy', valid_feature)
np.save(f'Data/{load_data_path}/train_label.npy', train_label)
np.save(f'Data/{load_data_path}/valid_label.npy',valid_label)
np.save(f'Data/{load_data_path}/test_feature.npy',test_feature)
np.save(f'Data/{load_data_path}/test_label.npy',test_label)

In [ ]:
# train_feature = np.load(f'Data/{load_data_path}/train_feature.npy')
# valid_feature = np.load(f'Data/{load_data_path}/valid_feature.npy')
# test_feature = np.load(f'Data/{load_data_path}/test_feature.npy')
# train_label = np.load(f'Data/{load_data_path}/train_label.npy')
# valid_label = np.load(f'Data/{load_data_path}/valid_label.npy')
# test_label = np.load(f'Data/{load_data_path}/test_label.npy')

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

def eval_metrics(classifier, test_features, test_labels):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    
    base_score   = classifier.score(test_features, test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    
    target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions, target_names=target_names),"\n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Avarge Precision: ", av_precision)
    
    return base_score,accuracy,av_precision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb

Models = { 
    "Decision Tree": DecisionTreeClassifier(),      
    "KNearest": KNeighborsClassifier(n_jobs=-1),   
    "XGB" : xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0,
                            #   max_depth=10, n_estimators=20, learning_rate=0.05
                              ), 
    # "Extra_T" : ExtraTreesClassifier(n_estimators=50),
    # "R_forest" : RandomForestClassifier(n_estimators=50),       
}

In [ ]:
import mlflow
import mlflow.tensorflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [ ]:
counter = 1
for Model_Name, classifier in Models.items(): 
    # with mlflow.start_run(nested=True):
    print(f"{counter}. {Model_Name}")
    
    with mlflow.start_run():
        # fit the model
        from joblib import parallel_backend
        with parallel_backend('threading', n_jobs=-1):
            classifier.fit(train_feature, train_label)
        counter = counter + 1
        
        # Calculate the metrics
        base_score,accuracy,av_precision = eval_metrics(classifier,
                                                        valid_feature,
                                                        valid_label)  
        
        mlflow.log_param("Model"           , Model_Name)
        mlflow.log_param("Dataset" , f'{load_data_path}')
        mlflow.log_metric("base_score"     , base_score)
        mlflow.log_metric("accuracy"       , accuracy)
        mlflow.log_metric("av_precision"   , av_precision)
        
        if av_precision > 0.95 :
            # mlflow.sklearn.log_model(classifier,Model_Name, signature=signature)
            print(f"f1 socre is more than 0.945 so the {Model_Name} is saved")
        else :
            print(f"Because f1 socre is not quality. The model is skip to saving phase.")
        
        print("________________________________________")

### Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime

In [ ]:
epoch = 5
input_shape = train_feature.shape[1]

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(inputs)
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x) # stacking RNN layers requires return_sequences=True
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dense(64, activation="relu")(x) 
x = layers.Dense(16, activation="relu")(x) 
x = layers.Dense(8, activation="relu")(x) 
outputs = layers.Dense(4, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")



model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()

In [ ]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_feature, train_label))
train_dataset =  train_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_feature, valid_label))
valid_dataset = valid_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_feature, test_label))
test_dataset = test_dataset.batch(512).prefetch(tf.data.AUTOTUNE)

train_dataset, valid_dataset, test_dataset

In [ ]:
# fit the model
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42

start = datetime.now()
with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model.fit(train_dataset,
                            batch_size=2048,
                            steps_per_epoch=len(train_dataset),
                            validation_data=valid_dataset,
                            validation_steps=int(len(valid_dataset)),
                            callbacks=[early_stopping, reduce_lr],
                            epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model.evaluate(test_dataset)
    
    mlflow.log_param("Model"   , "LSTM")
    mlflow.log_param("Dataset" , f'{load_data_path}')
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()
    
    # Calculate the metrics
    model_preds_probs   = model.predict(test_feature)
    av_precision    = average_precision_score(test_label, model_preds_probs)
    print(av_precision)
    mlflow.log_metric("av_precision"   , av_precision)

print("________________________________________")

In [ ]:
# Build the model 
input_shape = 4

inputs = layers.Input(shape=input_shape, name="input_layer")
x = layers.Dense(32, activation ='relu')(inputs)
x = layers.Dense(64, activation = 'relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation ='relu')(x)
x = layers.Dense(512, activation = 'relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(1024, activation ='relu')(x)
x = layers.Dense(512, activation = 'relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax",name="output_layer")(x)      
model_DNN = tf.keras.Model(inputs, outputs) 

model_DNN.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model_DNN.summary()



In [ ]:
# Fit the model
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.set_seed = 42
np.random.seed(40)
counter = 1
epoch = 5
Model_Name = 'DNN_model'


with mlflow.start_run():
    mlflow.tensorflow.autolog()
    
    # fit the model
    # with tf.device('/GPU:0'):
    start = datetime.now()
    history_model = model_DNN.fit(train_dataset,
                                  batch_size=2048,
                                  steps_per_epoch=len(train_dataset),
                                  validation_data=valid_dataset,
                                  validation_steps=int(len(valid_dataset)),
                                  callbacks=[early_stopping, reduce_lr],
                                  epochs=epoch) 
    end = datetime.now()
    print(f"The time taken to train the model is {end - start}")
        
    # Evaluate model
    model_DNN.evaluate(test_dataset)
    
    # Calculate the metrics
    # model_results = eval_metrics(model, 
    #                              test_features = test_feature,
    #                              test_labels = test_label)
    
    mlflow.log_param("Model"           , Model_Name)
    mlflow.log_param("Dataset" , f'{load_data_path}')
    # mlflow.log_params(model_results)
    mlflow.tensorflow.autolog()
    
    # Calculate the metrics
    model_preds_probs   = model_DNN.predict(test_feature)
    av_precision    = average_precision_score(test_label, model_preds_probs)
    print(av_precision)
    mlflow.log_metric("av_precision"   , av_precision)
    
    
    print("________________________________________")